In [1]:
import ray, random, os 
ray.shutdown()

In [3]:
import ray, random, os 
from ray.air.config import ScalingConfig
import pandas as pd
ray.init(logging_level="ERROR",ignore_reinit_error=True, _temp_dir='/Volumes/ssdmac/ray/')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from worker_standlone import WorkerStandAlone
from multi_agent import MultiAgent
from manager import Manager
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.rllib.utils import check_env
from ray.tune.logger import pretty_print

In [3]:
from ray.rllib.utils import check_env
check_env(MultiAgent())

(raylet) [2023-10-04 16:01:59,809 E 63471 4605788] (raylet) file_system_monitor.cc:111: /Users/floriankockler/raytemp/session_2023-10-04_16-01-47_319714_63420 is over 95% full, available space: 11662782464; capacity: 245107195904. Object creation will fail if spilling is required.


In [ ]:
multi_agent_env = MultiAgent()

# Initialize environment
obs, info = multi_agent_env.reset()

# Loop for a maximum of 6310 steps or until done
for step in range(6310):
    actions = {}
    
    # Collect sample actions for each agent
    for agent_id, agent_obs in obs.items():
        action_space = multi_agent_env.action_space[agent_id]
        action = action_space.sample()
        actions[agent_id] = action
    
    # Step the environment
    obs, reward, done, _, info = multi_agent_env.step(actions)
    
    # Check if any agent is done and reset the environment if so
    if done["__all__"]:
        obs, info = multi_agent_env.reset()

In [ ]:
multi_agent_env = MultiAgent()


for episode in range(1):
    obs, info = multi_agent_env.reset()
    done = {"__all__": False}
    
    print(f"Episode {episode + 1}")

    while not done["__all__"]:
        actions = {}
        
        # Collect actions for each agent
        for agent_id, agent_obs in obs.items():
            action_space = multi_agent_env.action_space[agent_id]
            action = action_space.sample()
            actions[agent_id] = action

        # Step the environment
        obs, reward, done, _, info = multi_agent_env.step(actions)

        # print(f"Actions: {actions}")
        # print(f"Observations: {obs}")
        # print(f"Rewards: {reward}")
        # print(f"Done flags: {done}")
        # print(f"Info: {info}")

    print("Episode done!")

In [4]:
if os.name == 'nt':
    path_to_save = "C:\\GitHub\\ray\\workertune"
else:
    path_to_save = "/Volumes/SSD980/ray/trial2/"


env = MultiAgent()
def env_creator(env_config):
    return MultiAgent()  

register_env("MultiAgent", env_creator)

asha_scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=1000,
    grace_period=200,
    reduction_factor=3,
    brackets=1,
)

def create_policy_spec(agent_id):
    # print(f"Creating policy for {worker_id} with obs space {env.observation_space[worker_id]} and action space {env.action_space[worker_id]}")
    return PolicySpec(
        observation_space=env.observation_space[agent_id],
        action_space=env.action_space[agent_id],
        config={}
    )

controller_policy_spec = PolicySpec(
    observation_space=env.observation_space['controller'],
    action_space=env.action_space['controller'],
    config={}
)

policies = {
    "controller_policy": controller_policy_spec,
}

for agent_id in env.agents:
    policies[agent_id] = create_policy_spec(agent_id)

def policy_mapping_fn(agent_id, episode=None, agent=None, **kwargs):
    if agent_id == 'controller':
        # print(f"!!!!!! policy mapping manager: {agent_id}")
        return "controller_policy"
    elif agent_id in env.agents:
        return agent_id
    else:
        print("defaul policy triggered")
        return "default_policy"

param_space = {
     "env": "MultiAgent",
    "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "lr": tune.uniform(1e-5,1e-4),
        "gamma": tune.uniform(0.95, 0.9999),
        "lambda": tune.uniform(0.9,1.0),
        "entropy_coeff": tune.uniform(0.01,0.1),
        "vf_loss_coeff": tune.uniform(0.1,0.3),
        "num_workers": 6, 
        #Change for Debugging
        "log_level": "ERROR",
        "output": "logdir",
        "monitor": True,
}

analysis = tune.run(
    "A2C", 
    metric="episode_reward_mean", 
    num_samples=10,
    resume=False,
    mode="max",
    config=param_space, 
    local_dir=path_to_save,
    search_alg=None,
    scheduler=asha_scheduler,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=300),
    max_concurrent_trials=1,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 1,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
    )

print(pretty_print(analysis.last_result))
print("Best hyperparameters found were: ", pretty_print(analysis.best_config))

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(


== Status ==
Current time: 2023-10-04 17:51:37 (running for 00:00:00.43)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 600.000: None | Iter 200.000: None
Logical resource usage: 7.0/8 CPUs, 0/0 GPUs
Result logdir: /Volumes/SSD980/ray/trial2/A2C
Number of trials: 1/10 (1 PENDING)




(pid=10826) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=10826) 2023-10-04 17:51:44,030	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=10826) 2023-10-04 17:51:44,030	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=10826) 2023-10-04 17:51:44,030	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=10834) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=10835) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=10826) Trainable.setup took 13.753 seconds. If your trainable is 

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
A2C_MultiAgent_e6426_00000,10188864,"{'ObsPreprocessorConnector_ms': 0.019185730389186313, 'StateBufferConnector_ms': 0.0038927623203822543, 'ViewRequirementAgentConnector_ms': 0.08245149680546351}","{'num_env_steps_sampled': 727776, 'num_env_steps_trained': 727776, 'num_agent_steps_sampled': 10188864, 'num_agent_steps_trained': 10188864}",{},6305,{},-160.967,-1008.03,-4487.19,0,10188864,10188864,727776,128,0.137421,727776,128,0.137421,0,6,0,0,128,"{'cpu_util_percent': 73.53571428571429, 'ram_util_percent': 83.49285714285713}","{'controller_policy': 0.0, 'ABT.US': 0.0, 'AMGN.US': 0.0, 'BDX.US': -19.979345321655273, 'BMY.US': -4.0, 'HUM.US': -19.97932243347168, 'JNJ.US': 0.0, 'LLY.US': 0.0, 'MDT.US': 0.0, 'MRK.US': 0.0, 'PFE.US': 0.0, 'SYK.US': 0.0, 'TMO.US': -19.97937161181835, 'UNH.US': 0.0}","{'controller_policy': 0.0, 'ABT.US': -20.02922131538391, 'AMGN.US': -48.51601863861084, 'BDX.US': -153.62597711861673, 'BMY.US': -61.06236915120287, 'HUM.US': -73.10452107114241, 'JNJ.US': -25.481669235229493, 'LLY.US': -113.31748215691401, 'MDT.US': -106.78236386369448, 'MRK.US': -24.46366325378418, 'PFE.US': -24.862850346297638, 'SYK.US': -85.51504980735615, 'TMO.US': -197.86258909929336, 'UNH.US': -73.40641773178905}","{'controller_policy': 0.0, 'ABT.US': -444.8187770843506, 'AMGN.US': -1968.2212772369385, 'BDX.US': -927.4580383300781, 'BMY.US': -821.1488208770752, 'HUM.US': -642.5948664115276, 'JNJ.US': -475.5227279663086, 'LLY.US': -1860.6670837402344, 'MDT.US': -2153.991724014282, 'MRK.US': -482.0200958251953, 'PFE.US': -284.9890995025635, 'SYK.US': -1496.4380168914795, 'TMO.US': -2945.3641405562958, 'UNH.US': -2271.2881145477295}","{'mean_raw_obs_processing_ms': 14.853657637012816, 'mean_inference_ms': 38.84841537244145, 'mean_action_processing_ms': 2.5069739349320694, 'mean_env_wait_ms': 2.650537250390438, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 93358.513, 'sample_time_ms': 93246.83, 'learn_time_ms': 79.692, 'learn_throughput': 401.545, 'synch_weights_time_ms': 30.934}"
A2C_MultiAgent_e6426_00001,2729216,"{'ObsPreprocessorConnector_ms': 0.019688200164627242, 'StateBufferConnector_ms': 0.0031049434955303487, 'ViewRequirementAgentConnector_ms': 0.07707908913329407}","{'num_env_steps_sampled': 194944, 'num_env_steps_trained': 194944, 'num_agent_steps_sampled': 2729216, 'num_agent_steps_trained': 2729216}",{},6305,{},-5352.58,-25394.5,-63377,0,2729216,2729216,194944,800,79.7901,194944,800,79.7901,0,6,0,0,800,"{'cpu_util_percent': 89.37272727272727, 'ram_util_percent': 83.66363636363636}","{'controller_policy': 0.0, 'ABT.US': -33.98000526428223, 'AMGN.US': -31.98000717163086, 'BDX.US': -19.979995727539062, 'BMY.US': -19.979999542236328, 'HUM.US': -19.97998046875, 'JNJ.US': -661.443941116333, 'LLY.US': -19.979999542236328, 'MDT.US': -32.98000526428223, 'MRK.US': -117.01384544372559, 'PFE.US': -21.97998809814453, 'SYK.US': -20.97983741760254, 'TMO.US': -19.979984283447266, 'UNH.US': -30.980009078979492}","{'controller_policy': 0.0, 'ABT.US': -1302.2244727225088, 'AMGN.US': -4453.61256598212, 'BDX.US': -1628.2435304286487, 'BMY.US': -1463.189207656908, 'HUM.US': -801.6533746935622, 'JNJ.US': -2247.2317129901867, 'LLY.US': -1897.0697936264955, 'MDT.US': -2147.7699837039745, 'MRK.US': -1485.6286236652052, 'PFE.US': -1056.2969676477348, 'SYK.US': -2661.5139119156, 'TMO.US': -1363.429581851081, 'UNH.US': -2886.6

== Status ==
Current time: 2023-10-04 17:56:37 (running for 00:05:00.59)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 600.000: None | Iter 200.000: None
Logical resource usage: 7.0/8 CPUs, 0/0 GPUs
Result logdir: /Volumes/SSD980/ray/trial2/A2C
Number of trials: 1/10 (1 RUNNING)


(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) Worker done
(RolloutWorker pid=10834) =========HRL is done=============
(RolloutWorker pid=10834) HRL is done
(RolloutWorker pid=10834) day: 6305, episode: 2
(RolloutWorker pid=10834) Controller invalid actions: 
(RolloutWorker pid=

...

== Status ==
Current time: 2023-10-05 06:27:22 (running for 12:35:45.54)
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 600.000: -6909.540991614746 | Iter 200.000: -13687.777418562755
Logical resource usage: 7.0/8 CPUs, 0/0 GPUs
Current best trial: e6426_00000 with episode_reward_mean=-1008.030192789315 and parameters={'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


== Status ==
Current time: 2023-10-05 06:42:45 (running for 12:51:08.78)
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 600.000: -6909.540991614746 | Iter 200.000: -13687.777418562755
Logical resource usage: 7.0/8 CPUs, 0/0 GPUs
Current best trial: e6426_00000 with episode_reward_mean=-1008.030192789315 and parameters={'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


In [ ]:
env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)



analysis = tune.run(
        "A2C",
        metric="episode_reward_mean",
        mode="max",
        config={
            "env": "worker",
            "env_config": {"initial_capital": 1e6},
            "lr": tune.uniform(1e-5, 1e-4),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
}, )

In [ ]:


n_iterations = 30000


env = WorkerStandAlone(data=data)

state = env.reset()

for _ in range(n_iterations):
    action = env.action_space.sample()  # Generate random actions for the manager
    worker_dones = {...}  # You need to provide the values for worker_dones
    worker_truncateds = {...}  # You need to provide the values for worker_truncateds

    obs, reward, done, truncated, info = env.step(action)

    if done:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:

env = HRL()

n_iterations = 1

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 



policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}

    
algo = PPOConfig().environment(env=HRL).multi_agent(
        policies=policies,
        policy_mapping_fn = policy_mapping_fn,  
    ).training(train_batch_size=4000).build()

print(algo.train())

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

analysis = tune.run("A2C", metric="episode_reward_mean", mode="max",config=param_space)

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
algo = PPOConfig().environment(env=HRL).multi_agent(
    policies={
        "policy_1": ()
    }
)

In [ ]:
from ray.tune import CLIReporter
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
env = HRL()

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune_config,
    param_space=param_space,

    run_config=run_config,
)
results = tuner.fit()


In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()